### Team 2: China's K12 Online Education Data Management System

In [ ]:
import pandas as pd
import numpy as np

### Online Platform Table

In [ ]:
online_platform = pd.DataFrame(np.array([[1, "yuanfudao", "https://www.yuanfudao.com/"], 
                                         [2, "genshuixue","https://www.genshuixue.com/"], 
                                         [3, "youdao", "https://ke.youdao.com/"]]),
                               columns=['platform_id', 'platform_name', 'platform_url'])

online_platform

,platform_id,platform_name,platform_url
0,1,yuanfudao,https://www.yuanfudao.com/
1,2,genshuixue,https://www.genshuixue.com/
2,3,youdao,https://ke.youdao.com/


In [ ]:
online_platform.to_csv("/content/drive/MyDrive/INFO7275_Advanced_DB/platform_list.csv",index= False)

##1） Yuanfudao  Online Platform

In [ ]:
grade_rawdata_yuanfudao = pd.read_excel("/content/drive/MyDrive/INFO7275_Advanced_DB/yuanfudao_grade.xlsx")

In [ ]:
grade_yuanfudao = pd.DataFrame(grade_rawdata_yuanfudao)
grade_yuanfudao = grade_yuanfudao[["grade_id",'grade_name']]
grade_yuanfudao = grade_yuanfudao.drop_duplicates('grade_id')
grade_yuanfudao = grade_yuanfudao[~grade_yuanfudao.grade_name.isin(['中班','大班'])].sort_values('grade_id')
print(grade_yuanfudao.grade_name.unique())
grade_yuanfudao

['一年级' '二年级' '三年级' '四年级' '五年级' '六年级' '七年级' '八年级' '九年级' '高一' '高二' '高三']


,grade_id,grade_name
9,1,一年级
8,2,二年级
7,3,三年级
6,4,四年级
5,5,五年级
4,6,六年级
13,7,七年级
12,8,八年级
11,9,九年级
3,10,高一


In [ ]:
grade_yuanfudao.to_excel('/content/drive/MyDrive/INFO7275_Advanced_DB/dataset/grade_list.xlsx',index= False)

In [ ]:
yuanfudao_course = pd.read_excel("/content/drive/MyDrive/INFO7275_Advanced_DB/yuanfudao_course.xlsx")
yuanfudao_course.head(3)

,group_id,grade_id,grade_name,course_id,course_name,course_type,product_id,variant_id,price,quantity,sold_count,pre_sale_time,start_sale_time,end_sale_time,original_price,left_count,sold_out,start_time,end_time,display_start_time,display_end_time,sub_name,recommended,as_group,label,teacher,subject,episode_count,run_id,__id
0,NaN,99,中班,5078419.0,幼儿园中班数学能力与专注力春季系统班（周五、周日19:30）,lesson,19654271,14378097,1400,33,32,2020-11-11,2020-11-11,2021-04-05,0,1,0,2021-03-05,2021-06-13,2021-06-13,2021-06-13,3月5日-6月13日 · 每周五日19:30上课,0,0,系统班,李蒙蒙|张思思,数学,44,2021-03-21,711438
1,NaN,99,中班,4943315.0,幼儿园中班数学能力与专注力春季系统班（周四、周日18:30）,lesson,19516905,16714371,1400,30,23,2020-11-11,2020-11-11,2021-04-05,0,7,0,2021-03-04,2021-06-13,2021-06-13,2021-06-13,3月4日-6月13日 · 每周四日18:30上课,0,0,系统班,刘美辰|金菊,数学,44,2021-03-21,711437
2,NaN,99,中班,4943313.0,幼儿园中班数学能力与专注力春季系统班（周一、周四18:30）,lesson,19516903,15079013,1400,32,32,2020-11-11,2020-11-11,2021-04-05,0,0,1,2021-03-01,2021-06-10,2021-06-10,2021-06-10,3月1日-6月10日 · 每周一四18:30上课,0,0,系统班,陈静仪|陈缘,数学,44,2021-03-21,711436


In [ ]:
yuanfudao_course = pd.read_excel("/content/drive/MyDrive/INFO7275_Advanced_DB/yuanfudao_course.xlsx")
yuanfudao_course.head(3)
subject_yuanfudao = pd.DataFrame(yuanfudao_course.subject.unique())
subject_yuanfudao.reset_index(inplace=True)
subject_yuanfudao = subject_yuanfudao.rename(columns={"index": "subject_id",0:"subject_name"})
subject_yuanfudao

,subject_id,subject_name
0,0,数学
1,1,政治
2,2,地理
3,3,历史
4,4,生物
5,5,化学
6,6,物理
7,7,英语
8,8,语文


Teacher data cleaning:

In [ ]:
teacher_yuanfudao_rawdata = pd.read_excel('/content/drive/MyDrive/INFO7275_Advanced_DB/dataset/yuanfudao_course_2020.xlsx')
teacher_yuanfudao = pd.DataFrame(teacher_yuanfudao_rawdata)[['course_id','teacher']]
teacher_yuanfudao[['teacher1','teacher2']] = teacher_yuanfudao.teacher.str.split("|",expand = True).iloc[:,0:2]
teacher_yuanfudao = teacher_yuanfudao.drop(columns = 'teacher')

In [ ]:
teacher_list=  pd.DataFrame(np.concatenate((teacher_yuanfudao.iloc[:,[0,1]],teacher_yuanfudao.iloc[:,[0,2]])),columns = ['course_id','teacher'])
teacher_list = teacher_list[~teacher_list.teacher.isin(['班主任','主讲老师'])]
teacher_list = pd.DataFrame(teacher_list)[['teacher']].drop_duplicates('teacher')
teacher_list = teacher_list.drop_duplicates('teacher')
teacher_list.reset_index(inplace = True)
teacher_list = teacher_list.rename(columns={"index": "teacher_id","teacher":"teacher_name"})

In [ ]:
teacher_list.to_excel('/content/drive/MyDrive/INFO7275_Advanced_DB/dataset/teacher_info.xlsx',index = False)

In [ ]:
teacher_list.head(3)

,teacher_id,teacher_name
0,0,刘薇
1,1,程磊
2,2,严攀


Sales table

In [ ]:
rawdata = pd.read_excel('/content/drive/MyDrive/INFO7275_Advanced_DB/dataset/yuanfudao_course_2020.xlsx')
rawdata.head(3)


,group_id,grade_id,grade_name,course_id,course_name,course_type,product_id,variant_id,price,quantity,sold_count,pre_sale_time,start_sale_time,end_sale_time,original_price,left_count,sold_out,start_time,end_time,display_start_time,display_end_time,sub_name,recommended,as_group,label,teacher,subject,episode_count,run_id,__id
0,18657.0,11,高二,NaN,高二语文春季系统班-正在报名,bundleSystematic,0,0,599,14000,5181,NaN,NaN,2020-04-06,0,0,0,2020-02-23,2020-06-14,2020-06-14,2020-06-14,2月23日-6月7日 · 周日13:00上课,0,0,系统班,刘薇,语文,23.0,2020-02-15,303800
1,19595.0,11,高二,NaN,【目标清北班】高二数学春季系统班-正在报名,bundleSystematic,0,0,599,2000,1506,NaN,NaN,2020-04-06,0,0,0,2020-02-22,2020-06-13,2020-06-13,2020-06-13,2月23日-6月7日 · 周六21:30上课,0,0,系统班,程磊,数学,23.0,2020-02-15,303801
2,19597.0,11,高二,NaN,高二数学春季系统班-正在报名,bundleSystematic,0,0,599,22000,12750,NaN,NaN,2020-04-06,0,0,0,2020-02-22,2020-06-13,2020-06-13,2020-06-13,2月22日-6月6日 · 周六19:00上课,0,0,系统班,严攀,数学,23.0,2020-02-15,303802


In [ ]:
rawdata = pd.DataFrame(rawdata)
sales_data = rawdata[['course_id','price','original_price','quantity','sold_count','pre_sale_time','start_sale_time','end_sale_time','left_count','sold_out']]
sales_data = sales_data[pd.notnull(sales_data["course_id"])]
sales_data = sales_data.drop_duplicates('course_id')

In [ ]:
sales_data.to_excel('/content/drive/MyDrive/INFO7275_Advanced_DB/dataset/sales_info.xlsx',index = False)

Course data cleaning

In [275]:
course_yuanfudao = pd.DataFrame(rawdata)[['grade_id','grade_name','course_id','course_name','subject','start_time','end_time','episode_count','run_id']]
course_yuanfudao.rename(columns={"episode_count": "lesson_num","subject":"subject_name"},inplace= True)

In [276]:
course_yuanfudao = course_yuanfudao[pd.notnull(course_yuanfudao["course_id"])]
course_yuanfudao['run_id'] = pd.to_datetime(course_yuanfudao['run_id']).astype(str)
course_yuanfudao['month'] = course_yuanfudao["run_id"].str[0:7]
#remove duplicated data 
course_yuanfudao = course_yuanfudao.drop_duplicates(['course_id','month'])
course_yuanfudao = course_yuanfudao.drop(columns='month')

In [277]:
course_yuanfudao.head()

,grade_id,grade_name,course_id,course_name,subject_name,start_time,end_time,lesson_num,run_id
46,7,七年级,3511191.0,初一语文题型技巧特训班,语文,2020-03-07,2020-03-10,7.0,2020-02-15
47,7,七年级,3163273.0,初一语文春季系统班（周五18:00）,语文,2020-02-21,2020-06-19,36.0,2020-02-15
48,7,七年级,3163275.0,初一语文春季系统班（周六09:00）,语文,2020-02-22,2020-06-20,36.0,2020-02-15
49,7,七年级,3163277.0,初一语文春季系统班（周六14:00）,语文,2020-02-22,2020-06-20,36.0,2020-02-15
50,7,七年级,3163279.0,初一语文春季系统班（周六14:00）,语文,2020-02-22,2020-06-20,36.0,2020-02-15


In [278]:
#course vs teacher_id
course_teacher =  pd.DataFrame(np.concatenate((teacher_yuanfudao.iloc[:,[0,1]],teacher_yuanfudao.iloc[:,[0,2]])),columns = ['course_id','teacher'])
course_teacher = course_teacher[pd.notnull(course_teacher['course_id'])]
course_teacher = course_teacher[~course_teacher.teacher.isin(['班主任','主讲老师'])]
course_teacher = course_teacher.drop_duplicates('course_id')
course_teacher =  pd.merge(left = course_teacher, right = teacher_list, how = 'left', left_on='teacher', right_on='teacher_name')
course_teacher = course_teacher[['course_id','teacher_id']]
course_teacher.head(3)

,course_id,teacher_id
0,3.51119e+06,46
1,3.16327e+06,46
2,3.16328e+06,46


In [279]:
#merge course_info and course_teacher
course_info = pd.merge(left = course_yuanfudao, right = course_teacher, how = 'left', on = 'course_id')

In [280]:
course_info.head(3)

,grade_id,grade_name,course_id,course_name,subject_name,start_time,end_time,lesson_num,run_id,teacher_id
0,7,七年级,3.51119e+06,初一语文题型技巧特训班,语文,2020-03-07,2020-03-10,7.0,2020-02-15,46.0
1,7,七年级,3.16327e+06,初一语文春季系统班（周五18:00）,语文,2020-02-21,2020-06-19,36.0,2020-02-15,46.0
2,7,七年级,3.16328e+06,初一语文春季系统班（周六09:00）,语文,2020-02-22,2020-06-20,36.0,2020-02-15,46.0


In [282]:
#merge with subject
subject_info = pd.read_excel('/content/drive/MyDrive/INFO7275_Advanced_DB/dataset/subject_list.xlsx')
subject_info = subject_info[['subject_id','subject_name']]
subject_info.head(3)
#merge course_info and course_teacher
course_info = pd.merge(left = course_info, right = subject_info, how = 'left', on = 'subject_name')

In [283]:
course_info.head(3)

,grade_id,grade_name,course_id,course_name,subject_name,start_time,end_time,lesson_num,run_id,teacher_id,subject_id
0,7,七年级,3.51119e+06,初一语文题型技巧特训班,语文,2020-03-07,2020-03-10,7.0,2020-02-15,46.0,8.0
1,7,七年级,3.16327e+06,初一语文春季系统班（周五18:00）,语文,2020-02-21,2020-06-19,36.0,2020-02-15,46.0,8.0
2,7,七年级,3.16328e+06,初一语文春季系统班（周六09:00）,语文,2020-02-22,2020-06-20,36.0,2020-02-15,46.0,8.0


In [290]:
course_info['platform_id'] = 1
course_info = course_info.drop(columns = ['grade_id','grade_name'])

In [291]:
course_info.head(3)

,grade_id,course_id,course_name,start_time,end_time,lesson_num,run_id,teacher_id,subject_id,platform_id
0,7,3.51119e+06,初一语文题型技巧特训班,2020-03-07,2020-03-10,7.0,2020-02-15,46.0,8.0,1
1,7,3.16327e+06,初一语文春季系统班（周五18:00）,2020-02-21,2020-06-19,36.0,2020-02-15,46.0,8.0,1
2,7,3.16328e+06,初一语文春季系统班（周六09:00）,2020-02-22,2020-06-20,36.0,2020-02-15,46.0,8.0,1


In [292]:
course_info.to_excel("/content/drive/MyDrive/INFO7275_Advanced_DB/dataset/course_yuanfudao_info.xlsx",index = False)